# Lamin Compute with Modal Backend

We show how to run compute jobs with Modal backend to run any script in the cloud without having to manage compute resources.

There are three major components to run compute jobs seemlessly in the cloud.

1. Access to your Artifacts and Data - (Lamin) 
2. Seemless access to a compute backend to run any workload - (Modal)
3. Tracking your workload inputs and outputs - (Lamin)

In [2]:
# !pip install modal 'lamindb[jupyter]'
!modal setup

The web browser should have opened for you to authenticate and get an API token.
If it didn't, please copy this URL into your web browser manually:

]8;id=311415;https://modal.com/token-flow/tf-3hZMeO36QqLSRCpsT0RADS\https://modal.com/token-flow/tf-3hZMeO36QqLSRCpsT0RADS]8;;\

⠙ Waiting for authentication in the web browser
⠇ Waiting for token flow to complete...omplete...
Web authentication finished successfully!
Token is connected to the ragyhaddad workspace.
Verifying token against https://api.modal.com
Token verified successfully!
⠋ Storing token
Token written to /Users/rhaddad/.modal.toml in profile ragyhaddad.


## Define your script

In [ ]:
import os
import lamindb as ln

API_KEY = os.environ['lamin_user_api_key']
PROJECT_NAME = os.environ['lamin_project_name']

# LAMIN SETUP
ln.setup.login(api_key=API_KEY)
ln.connect('laminlabs/lamindata')
my_project = ln.Project(name=PROJECT_NAME).save()


ln.track(project=PROJECT_NAME)

## Code Start 
def say_hello():
    print('Hello, World! lamin, user key has been passed successfully')


if __name__ == '__main__':
    say_hello()

## Code End
ln.finish()

we save the above script as `./helloworld.py`

Execute the script in the cloud by running:
```bash 
lamin run ./helloworld.py --project modal_project
``` 

In [ ]:
!lamin run ./helloworld.py --project modal_project

In [ ]:
import os
import lamindb as ln

API_KEY = os.environ['lamin_user_api_key']
PROJECT_NAME = os.environ['lamin_project_name']

# LAMIN SETUP
ln.setup.login(api_key=API_KEY)
ln.connect('laminlabs/lamindata')
ln.track()  # track your run of a notebook or script 

def main():
    # Access inputs -------------------------------------------
    artifact = ln.Artifact.using("laminlabs/cellxgene").get("7dVluLROpalzEh8m")  # query the artifact https://lamin.ai/laminlabs/cellxgene/artifact/7dVluLROpalzEh8m
    adata = artifact.load()[:, :100]  # load into memory or sync to cache: filepath = artifact.cache()
    
    # Your transformation -------------------------------------
    
    import scanpy as sc  # find marker genes with Scanpy
    
    sc.pp.normalize_total(adata)
    sc.pp.log1p(adata)
    sc.tl.rank_genes_groups(adata, groupby="cell_type")
    
    # Save outputs --------------------------------------------
    
    ln.Artifact.from_anndata(adata, key="my-datasets/my-result.h5ad").save()  # save versioned output

if __name__ == '__main__':
    main()
ln.finish()  # finish the run, save source code & run report

Save your script locally `./lamin_sc.py`

Then execute your script using 
```
lamin run lamin_sc.py --project lamin_sc --packages scanpy
```

## Specifying dependancies and images

A key component to run compute workloads is access to defining images and environments required to run your code. Below we go over examples where we include custom dependancies and images to the environment to run your code. 

### Save your local script

Below we write a pytorch script and add GPU access to it

In [ ]:
import os
import lamindb as ln

API_KEY = os.environ['lamin_user_api_key']
PROJECT_NAME = os.environ['lamin_project_name']

# LAMIN SETUP
ln.setup.login(api_key=API_KEY)
ln.connect('laminlabs/lamindata')
my_project = ln.Project(name=PROJECT_NAME).save()

ln.track(project=PROJECT_NAME)

def say_hello():
    import torch
    print('Imported pytorch and detecting GPUs')
    print(torch.cuda.is_available())


if __name__ == '__main__':
    say_hello()

ln.finish()

We save the above script locally as `./helloworld_gpu.py` 

To execute your code in the cloud run the following command
``` bash 
lamin run ./helloworld_gpu.py --project lamin_project_gpu --image nvcr.io/nvidia/pytorch:22.12-py3 --packages torch,numpy --gpu T4:1
```

Notice in the above command we specify the `--image` and we additionally add GPU access by passing `--gpu` flag where we specify `T4:1` as the GPU type and number of GPUs to attach to our compute job.

We can also specify additional `pip` dependancies via `--packages` `torch,numpy,seaborn`  